<a href="https://colab.research.google.com/github/BurningFalls/nlp-study/blob/main/BERT%20%EC%8B%A4%EC%8A%B5/BERT_KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B0%9C%EC%B2%B4%EB%AA%85_%EC%9D%B8%EC%8B%9D(Named_Entity_Recognition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.3 MB/s eta 0:00:00


In [2]:
pip install seqeval


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=db2675b559d863999fbc8f876ba74b26a9a236573aef8084cb5576faccd97ec8
  Stored in directory: /root/.cache/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built seqeval


# 1. 데이터 로드


In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_" -O ner_train_data.csv && rm -rf /tmp/cookies.txt


--2023-03-22 12:58:09--  https://docs.google.com/uc?export=download&confirm=&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_
Resolving docs.google.com (docs.google.com)... 142.250.125.101, 142.250.125.102, 142.250.125.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n281seirc8r72p6ni3sokfslathhltqr/1679489850000/17609157229046208934/*/1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_?e=download&uuid=3753cb72-d54e-484f-b8f9-14f49ac80a5d [following]
--2023-03-22 12:58:10--  https://doc-00-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n281seirc8r72p6ni3sokfslathhltqr/1679489850000/17609157229046208934/*/1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_?e=download&uuid=3753cb72-d54e-484f-b8f9-14f49ac80a5d
Resolving doc-00-7k-docs.googleusercontent.com (doc-00-7k-docs.googleusercontent.com)... 74.125.124.13

In [5]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3" -O ner_test_data.csv && rm -rf /tmp/cookies.txt


--2023-03-22 12:58:12--  https://docs.google.com/uc?export=download&confirm=&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3
Resolving docs.google.com (docs.google.com)... 142.250.125.101, 142.250.125.102, 142.250.125.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ogdloe1i1netmlu3rvek9q6i3ken307t/1679489850000/17609157229046208934/*/1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3?e=download&uuid=c0b2a862-7fe4-4360-94b2-4ee163e07627 [following]
--2023-03-22 12:58:13--  https://doc-10-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ogdloe1i1netmlu3rvek9q6i3ken307t/1679489850000/17609157229046208934/*/1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3?e=download&uuid=c0b2a862-7fe4-4360-94b2-4ee163e07627
Resolving doc-10-7k-docs.googleusercontent.com (doc-10-7k-docs.googleusercontent.com)... 74.125.124.13

In [6]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ" -O ner_label.txt && rm -rf /tmp/cookies.txt


--2023-03-22 12:58:14--  https://docs.google.com/uc?export=download&confirm=&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ
Resolving docs.google.com (docs.google.com)... 142.250.125.101, 142.250.125.102, 142.250.125.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k3f1i13ukq6fjo1o9se8vsrgmrma1kcm/1679489850000/17609157229046208934/*/1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ?e=download&uuid=5245a8e9-68f2-4983-87ce-1088885d7b63 [following]
--2023-03-22 12:58:15--  https://doc-00-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k3f1i13ukq6fjo1o9se8vsrgmrma1kcm/1679489850000/17609157229046208934/*/1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ?e=download&uuid=5245a8e9-68f2-4983-87ce-1088885d7b63
Resolving doc-00-7k-docs.googleusercontent.com (doc-00-7k-docs.googleusercontent.com)... 74.125.124.13

In [7]:
train_ner_df = pd.read_csv("ner_train_data.csv")

In [8]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [9]:
test_ner_df = pd.read_csv("ner_test_data.csv")

In [10]:
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


In [11]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [12]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [13]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [14]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 29


In [15]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [16]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [17]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)


100%|██████████| 81000/81000 [01:18<00:00, 1037.49it/s]


In [18]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)


100%|██████████| 9000/9000 [00:05<00:00, 1720.04it/s]


In [19]:
from transformers import TFBertForTokenClassification

In [20]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [21]:
with strategy.scope():
  model = TFBertForTokenClassification.from_pretrained("klue/bert-base", num_labels=tag_size, from_pt=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=model.hf_compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):
      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted.logits, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))
     

In [23]:
f1_score_report = F1score(X_test, y_test)

In [24]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3
   6/2532 [..............................] - ETA: 3:10 - loss: 2.3839

282/282 [==============================] - 19s 50ms/step
 - f1: 84.77
              precision    recall  f1-score   support

         AFW       0.61      0.60      0.60       394
         ANM       0.76      0.70      0.73       701
         CVL       0.78      0.86      0.81      5758
         DAT       0.90      0.93      0.92      2521
         EVT       0.74      0.76      0.75      1094
         FLD       0.67      0.61      0.63       228
         LOC       0.87      0.82      0.85      2126
         MAT       0.33      0.08      0.13        12
         NUM       0.91      0.92      0.92      5590
         ORG       0.84      0.88      0.86      4086
         PER       0.86      0.90      0.88      4426
         PLT       0.46      0.18      0.26        34
         TIM       0.80      0.92      0.86       314
         TRM       0.78      0.69      0.73      1964

   micro avg       0.84      0.86      0.85     29248
   macro avg       0.74      0.70      0.71     29248
weighted a

In [25]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [26]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 1088.36it/s]


In [27]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted.logits, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

In [28]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

In [29]:
test_samples = [sent1, sent2]

In [30]:
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2/2 [00:00<00:00, 837.10it/s]


1/1 [==============================] - 1s 551ms/step


In [31]:
result_list

[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]